In [13]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime as dt
from datetime import timedelta as td

In [14]:
# Adapter Layer

def read_csv_to_df(bucket, key, decoding = 'utf-8', sep = ','):
    csv_obj = bucket.Object(key=key).get().get('Body').read().decode(decoding)
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=sep)
    return df

def write_df_to_s3(bucket, df, key):
    out_buffer = BytesIO()
    df.to_parquet(out_buffer, index=False)
    bucket.put_object(Body=out_buffer.getvalue(), Key=key)
    return True

def list_files_in_prefix(bucket, prefix):
    files = [obj.key for obj in bucket.objects.filter(Prefix=prefix)]
    return files

In [15]:
# Application Layer

def extract(bucket, date_list):
    files = [key for date in date_list for key in list_files_in_prefix(bucket, date)]
    df = pd.concat([read_csv_to_df(bucket, obj) for obj in files], ignore_index=True)
    return df

def transform_report1(df, columns, arg_date):
    df = df.loc[:, columns]
    df.dropna(inplace=True)
    df['OpeningPrice'] = df.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')
    df['ClosingPrice'] = df.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['EndPrice'].transform('last')
    df = df.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('OpeningPrice', 'min'), 
                                                              closing_price_eur=('ClosingPrice', 'min'), 
                                                              minimum_price_eur=('MinPrice', 'min'),
                                                              maximum_price_eru=('MaxPrice','max'),
                                                              daily_traded_volume=('TradedVolume','sum'))
    df['previous_closing_price'] = df.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)
    df['%_change_closing_price'] = (df['closing_price_eur'] - df['previous_closing_price']) / df['previous_closing_price'] * 100
    df.drop(columns=['previous_closing_price'], inplace=True)
    df = df.round(decimals=2)
    df = df[df.Date >= arg_date]
    return df

def load(bucket, df, trg_key, trg_format):
    key = trg_key + dt.today().strftime("%Y%m%d_%H:%M:%S") + trg_format
    write_df_to_s3(bucket, df, key)
    return True

def etl_report1(src_bucket, trg_bucket, date_list, columns, arg_date, trg_key, trg_format):
    df = extract(src_bucket, date_list)
    df = transform_report1(df, columns, arg_date)
    load(trg_bucket, df, trg_key, trg_format)
    return True


In [16]:
# Application Layer - not core

def return_date_list(bucket, arg_date, src_format):
    min_date = dt.strptime(arg_date, src_format).date() - td(days=1)
    today = dt.today().date()
    return_date_list = [(min_date + td(days=x)).strftime(src_format) for x in range(0,(today - min_date).days + 1)]
    return return_date_list

In [17]:
# main function entrypoint

def main():
    
    # Parameters/Configurations
    # Later read config
    
    arg_date = '2022-12-28'
    src_format = '%Y-%m-%d'
    src_bucket = 'xetra-1234'
    trg_bucket = 'etl-project-data'
    columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
    trg_key = 'xetra_daily_report_' 
    trg_format = '.parquet'
    
    # Init
    s3 = boto3.resource('s3')
    bucket_src = s3.Bucket(src_bucket)
    bucket_trg = s3.Bucket(trg_bucket)
    
    # Run Application
    
    date_list = return_date_list(bucket_src, arg_date, src_format)
    etl_report1(bucket_src, bucket_trg, date_list, columns, arg_date, trg_key, trg_format)

In [18]:
# Call application run
main()

## Review contents of uploaded file to verify proper publishing

In [19]:
trg_bucket = 'etl-project-data'

s3 = boto3.resource('s3')
bucket_trg = s3.Bucket(trg_bucket)
    
for obj in bucket_trg.objects.all():
    print(obj.key)

xetra_daily_report_20230213_21:12:18.parquet
xetra_daily_report_20230213_21:40:47.parquet
xetra_daily_report_20230217_10:38:29.parquet
xetra_daily_report_20230217_10:39:41.parquet
xetra_daily_report_20230217_11:03:33.parquet


In [20]:
prq_obj = bucket_trg.Object(key='xetra_daily_report_20230217_10:39:41.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [21]:
df_report

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eru,daily_traded_volume,%_change_closing_price
0,AT000000STR1,2022-12-28,36.60,36.70,35.75,36.70,1773,-2.65
1,AT000000STR1,2022-12-29,36.60,36.70,35.75,36.70,1773,0.00
2,AT000000STR1,2022-12-30,36.60,36.70,35.75,36.70,1773,0.00
3,AT000000STR1,2022-12-31,36.60,36.70,35.75,36.70,1773,0.00
4,AT00000FACC2,2022-12-28,8.05,8.57,7.87,8.57,10205,2.51
...,...,...,...,...,...,...,...,...
12923,XS2376095068,2022-12-31,34.29,36.50,34.06,36.50,1000,0.00
12924,XS2434891219,2022-12-28,3.44,3.66,3.42,3.66,0,4.53
12925,XS2434891219,2022-12-29,3.44,3.66,3.42,3.66,0,0.00
12926,XS2434891219,2022-12-30,3.44,3.66,3.42,3.66,0,0.00
